<a href="https://colab.research.google.com/github/AndreSlavescu/Tensorflow-FreeCodeCamp/blob/main/Core-Learning-Algorithms/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup and Imports


In [55]:
!pip install -q sklearn

In [56]:
#line only required for notebooks
%tensorflow_version 2.x

In [57]:
from __future__ import division, absolute_import, print_function, unicode_literals

import tensorflow as tf

import pandas as pd

#Data

In [58]:
#Define constants for classification
CSV_COLUMN_NAMES = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"]
SPECIES = ["Setosa", "Versicolor", 'Virginica']

In [59]:
#Grab dataset with Keras to be read in a Pandas dataframe
train_path = tf.keras.utils.get_file(
    "iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv") #training data
test_path = tf.keras.utils.get_file(
    "iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv") #testing data

train = pd.read_csv(train_path, names = CSV_COLUMN_NAMES, header = 0)
test = pd.read_csv(test_path, names = CSV_COLUMN_NAMES, header = 0)

###Peek into data

In [60]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


###Use "species" as label

In [61]:
train_y = train.pop("Species")
test_y = test.pop("Species")

In [62]:
train.shape #120 entries with 4 features each

(120, 4)

#Input Function

In [63]:
def input_fn(features, labels, training = True, batch_size = 256):
  #convert inputs to dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #shuffle and repeat for training 
  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size) 

#Feature Columns

In [64]:
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


#Designing the Model

Example models for classification

*   ```DNNclassifier```
*   ```LinearClassifier```

A deep neural network (DNN) is more effective because there is no guarantee of a linear relation


In [65]:
#build a DNN with 2 hidden layers and 30 hidden nodes + 10 hidden nodes
classifier = tf.estimator.DNNClassifier(
    feature_columns = my_feature_columns,
    #two hidden layers of 30+10 hidden nodes
    hidden_units = [30,10],
    #the model has to choose between 3 classes
    n_classes = 3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpgexsh1zi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [66]:
#train the model
classifier.train(
    input_fn = lambda: input_fn(train,train_y,training=True), 
    steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpgexsh1zi/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1698046, step = 0
INFO:tensorflow:global_step/sec: 273.83
INFO:tensorflow:loss = 1.0654875, step = 100 (0.368 sec)
INFO:tensorflow:global_step/sec: 359.427
INFO:tensorflow:loss = 1.0491428, step = 200 (0.280 sec)
INFO:tensorflow:global_step/sec: 358.464
INFO:tensorflow:loss = 1.0210862, step = 300 (0.285 sec)
INFO:tensorflow:global_step/sec: 399.407
INFO:tensorflow:loss = 0.9893937, step = 400 (0.249 sec)
INFO:tensorflow:global_step/sec: 392.684
INFO:tensorflow:loss = 0.9787017, step = 500 (0.250 sec)
INFO:te

#Evaluate the Model

In [67]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test, test_y, training = False))
print("\nTest set accuracy: {accuracy:0.3f}\n".format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-07-15T05:03:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgexsh1zi/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27457s
INFO:tensorflow:Finished evaluation at 2021-07-15-05:03:10
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.475, average_loss = 0.69915617, global_step = 5000, loss = 0.69915617
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpgexsh1zi/model.ckpt-5000

Test set accuracy: 0.475



#Predictions

###prediction script that analyzes inputs and finds the best flower for the description

In [69]:
def input_fn(features, batch_size = 256):
  #convert inputs to a dataset without labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
predict = {}

print("Type float values for the following prompts")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dict in predictions:
  print(pred_dict)
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict["probabilities"][class_id]

  print('prediction is "{}" ({:.1f}%)'.format(
      SPECIES[class_id], 100 * probability))

Type numeric values for the following prompts
SepalLength: 2
SepalLength: 3
SepalLength: 4
SepalLength: 5
SepalLength: 2.3
SepalWidth: 2.3
PetalLength: 3.4
PetalWidth: 3.2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpgexsh1zi/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.8201818,  0.6144736,  2.05038  ], dtype=float32), 'probabilities': array([0.04377301, 0.18376777, 0.7724592 ], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
prediction is "Virginica" (77.2%)
